In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from pictionary_ai.params import *
from pictionary_ai.utils import *
from pictionary_ai.main.preprocessor import *
from pictionary_ai.model.models import *
from google.cloud import storage
from tqdm.auto import tqdm
from pathlib import Path
import random
import os
from sklearn.model_selection import train_test_split


In [8]:
def download_simplified_dataset(source_bucket:str = BUCKET_NAME_DRAWINGS_SIMPLIFIED, destination_path:str = LOCAL_DATA_PATH) -> None:
    '''
    Download the dataset on the machine for faster training.
    '''
    # Checking that the project's bucket matches the Google original one, if not copy Google data
    bucket_ready, reason = compare_buckets(BUCKET_NAME_DRAWINGS_SIMPLIFIED, ORIGINAL_BUCKET_DRAWINGS_SIMPLIFIED)
    if not bucket_ready:
        print(reason)
        copy_bucket(ORIGINAL_BUCKET_DRAWINGS_SIMPLIFIED, BUCKET_NAME_DRAWINGS_SIMPLIFIED)
    # Initialize a client
    storage_client = storage.Client()
    # Get the bucket
    bucket = storage_client.bucket(source_bucket)
    # List the blobs in the bucket (to tqdm to display progress)
    tqdm_blobs = tqdm(bucket.list_blobs())
    # Define the destination folder and create it if not existent
    destination_folder_path = f"{destination_path}/{source_bucket}"
    if not os.path.exists(destination_folder_path):
        os.makedirs(destination_folder_path)
    # Download all blobs to the destination folder
    for blob in tqdm_blobs[0:2]:
        destination_path = f"{destination_folder_path}/{blob.name}"
        blob.download_to_filename(destination_path)
    print(f"Downloaded the bucket {source_bucket} locally")

In [11]:
download_simplified_dataset(source_bucket='quickdraw_dataset/full/simplified',destination_path='/home/raj/code/rs-uk/pictionary-ai/raw_data')

NotFound: 404 GET https://storage.googleapis.com/storage/v1/b/quickdraw_dataset/full/simplified?projection=noAcl&prettyPrint=false: Not Found